In [2]:
import numpy as np
import matplotlib.pyplot as plt
import json
from ase.io import read
from matscipy.neighbours import neighbour_list
from collections import defaultdict

def dihedral(p1, p2, p3, p4):
    b0 = -(p2 - p1)
    b1 = p3 - p2
    b2 = p4 - p3
    b1 /= np.linalg.norm(b1)
    v = b0 - np.dot(b0, b1) * b1
    w = b2 - np.dot(b2, b1) * b1
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

def find_n_membered_rings(obj, n_number):
    result = []
    def _traverse(x):
        if isinstance(x, list):
            if all(isinstance(i, int) for i in x) and len(x) == n_number:
                result.append([i - 1 for i in x])
            else:
                for item in x:
                    _traverse(item)
    _traverse(obj)
    return result


def get_dumbbell_dihedrals_from_ring(atoms, ring_list, cutoff):
    i, j = neighbour_list('ij', atoms, cutoff=cutoff)
    nbrs = defaultdict(set)
    for atom_i, atom_j in zip(i, j):
        nbrs[atom_i].add(atom_j)
        nbrs[atom_j].add(atom_i)

    seen_pairs = set()
    angle_data = []
    
    for ring in ring_list:
        n = len(ring)
        for idx in range(n):
            a = ring[idx]
            d = ring[(idx + 1) % n]

            if (a, d) in seen_pairs or (d, a) in seen_pairs:
                continue

            if d not in nbrs[a] or len(nbrs[a]) != 3 or len(nbrs[d]) != 3:
                continue

            seen_pairs.add((a, d))

            others_a = sorted([n for n in nbrs[a] if n != d])
            others_d = sorted([n for n in nbrs[d] if n != a])
            
            b, c = others_a
            e, f = others_d

            p2 = atoms.positions[a]
            
            vec_ab = atoms.get_distance(a, b, vector=True, mic=True)
            vec_ac = atoms.get_distance(a, c, vector=True, mic=True)
            ag = vec_ab + vec_ac
            p1 = p2 + ag

            vec_ad = atoms.get_distance(a, d, vector=True, mic=True)
            p3 = p2 + vec_ad

            vec_de = atoms.get_distance(d, e, vector=True, mic=True)
            vec_df = atoms.get_distance(d, f, vector=True, mic=True)
            dh = vec_de + vec_df
            p4 = p3 + dh
            
            angle = dihedral(p1, p2, p3, p4)
            angle_data.append((a, b, c, d, e, f, angle))
            
    return angle_data



In [ ]:
atoms = read("a-As.extxyz")
with open("rings_out.json", "r") as f:
    data = json.load(f)

small_rings = []
large_rings = []
for n in range(3, 30):
    rings_n = find_n_membered_rings(data, n)
    if n < 7:
        small_rings.extend(rings_n)
    else:
        large_rings.extend(rings_n)

small_angles_data = get_dumbbell_dihedrals_from_ring(atoms, small_rings, cutoff=2.9)
large_angles_data = get_dumbbell_dihedrals_from_ring(atoms, large_rings, cutoff=2.9)

angles_small = [x[-1] for x in small_angles_data]
angles_large = [x[-1] for x in large_angles_data]

bins = np.linspace(-180, 180, 36)
h_small, _ = np.histogram(angles_small, bins=bins, density=True)
h_large, _ = np.histogram(angles_large, bins=bins, density=True)

diff = h_large - h_small
bin_centers = (bins[:-1] + bins[1:]) / 2

fig, ax = plt.subplots(figsize=(3.8, 2.2))

neg_mask = diff < 0
pos_mask = diff >= 0

ax.bar(bin_centers[neg_mask], diff[neg_mask], width=10, color='gray', alpha=0.7)
ax.bar(bin_centers[pos_mask], diff[pos_mask], width=10, color='purple', alpha=0.7)

ax.axhline(0, color='black', linestyle='--', linewidth=0.5)
ax.axvline(-90, linestyle='--', linewidth=0.5, color='gray')
ax.axvline(90, linestyle='--', linewidth=0.5, color='gray')

ax.set_xlabel("Dihedral angle (°)")
ax.set_ylabel("Distribution bias")

ax.set_xlim(-200, 200)
ax.set_ylim(-0.0018, 0.0018)

ax.set_xticks(np.arange(-180, 181, 90))
ax.set_yticks(np.arange(0, 0.003, 0.005))

plt.show()
